In [1]:
# Сверточные нейросети (CNN)

## 1. Что такое CNN
# Сверточная нейросеть (Convolutional Neural Network) — это тип нейросети, оптимизированный для работы с данными, где важна пространственная структура (например, изображение, звук, видео, сигнал).

# Главная идея — **извлечение признаков** через свертки (convolution), а не через полносвязные слои.

# ---

## 2. Основные блоки CNN

# Convolution (Conv2D) — фильтры, извлекающие признаки (края, текстуры, формы).  
# Параметры: размер ядра (3×3, 5×5), шаг (stride), отступ (padding), количество фильтров.

# Pooling (Max/Average) — уменьшает размерность, повышает устойчивость к сдвигам.  
# Параметры: размер окна и шаг.

# Activation (ReLU) — добавляет нелинейность, ускоряет обучение.  
# Популярные функции: ReLU, LeakyReLU, GELU.

# BatchNorm — стабилизирует распределение данных между слоями.  
# Dropout — борется с переобучением, случайно "выключая" нейроны.  
# Fully Connected (Dense) — заключительная часть, интерпретирует признаки и делает классификацию.

# ---

## 3. Как обучается CNN
# 1. Прямой проход (forward) — вычисляется предсказание.  
# 2. Функция потерь (loss) — измеряет ошибку (часто CrossEntropy).  
# 3. Обратное распространение (backpropagation) — корректирует веса.  
# 4. Оптимизаторы: Adam, SGD, RMSProp.

# ---

## 4. Ключевые архитектуры и их идеи

### LeNet-5 (1998)
# - Первая успешная CNN.
# - Conv → Pool → Conv → Pool → FC → Softmax.
# - Использовалась для распознавания цифр (MNIST).

### AlexNet (2012)
# - Прорыв на ImageNet.
# - Использовала ReLU, Dropout и GPU-обучение.
# - 5 сверток, 3 полносвязных слоя.

### VGG16 / VGG19 (2014)
# - Простая структура: блоки по 2–3 слоя 3×3 conv + pooling.
# - Очень глубокая, легко расширяемая.
# - Часто используется для feature extraction.

### GoogLeNet / Inception (2014)
# - В каждом блоке несколько параллельных сверток: 1×1, 3×3, 5×5.
# - Объединение разных масштабов признаков (Inception module).

### ResNet (2015)
# - Добавлены skip connections (остаточные связи): x + F(x).
# - Позволило обучать сети 100+ слоев без деградации.
# - Базовая архитектура для современных моделей.

### DenseNet (2016)
# - Каждый слой получает на вход результаты всех предыдущих.
# - Эффективное переиспользование признаков, меньше параметров.

### MobileNet (2017)
# - Легкая архитектура: depthwise + pointwise свертки.
# - Разработана для мобильных и встроенных систем.

### EfficientNet (2019)
# - Масштабирование по трем осям (глубина, ширина, разрешение).
# - Лучшая эффективность при меньших параметрах.

### ConvNeXt (2022)
# - CNN нового поколения, вдохновлена Vision Transformer.
# - LayerNorm, большие ядра, простая структура, но SOTA-результаты.

# ---

## 5. На чем обучали

# - Dataset: ImageNet (1.2 млн изображений, 1000 классов).  
# - Аппаратное обеспечение: GPU (NVIDIA V100 / A100).  
# - Фреймворки: PyTorch, TensorFlow, Keras.  
# - Loss-функция: CrossEntropy.  

# ---

## 6. Где используются CNN

# - Компьютерное зрение: классификация, детекция (YOLO, Faster R-CNN), сегментация (U-Net).  
# - Медицина: анализ рентгенов и МРТ.  
# - Автопилоты: обработка видеопотока с камер.  
# - Генеративные модели: GAN, Autoencoder.  
# - Аудио и временные ряды: CNN по спектрограммам.

# ---

## 7. Как использовать в PyTorch

# import torchvision.models as models
# import torch.nn as nn

# model = models.resnet50(weights='IMAGENET1K_V1')

# for param in model.parameters():
#     param.requires_grad = False

# model.fc = nn.Linear(2048, num_classes)

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch
import torch.nn as nn
from datetime import datetime as dt

In [3]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

In [4]:
all_data = torch.stack([img for img, _ in train_dataset], dim=0)

In [5]:
mean = all_data.mean()
std = all_data.std()
print(mean.item(), std.item())

0.13066047430038452 0.30810782313346863


In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [7]:
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [8]:
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(64*7*7, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
num_epochs = 6
time = dt.now()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, interval: {dt.now() - time}")

Epoch 1/6, Loss: 0.1758, interval: 0:00:33.015572
Epoch 2/6, Loss: 0.0529, interval: 0:01:06.796724
Epoch 3/6, Loss: 0.0362, interval: 0:01:40.555701
Epoch 4/6, Loss: 0.0275, interval: 0:02:14.209790
Epoch 5/6, Loss: 0.0196, interval: 0:02:47.376155
Epoch 6/6, Loss: 0.0173, interval: 0:03:20.664509


In [14]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 99.14%


In [15]:
print(f"{torch.__version__}, {torch.version.cuda}")

2.2.0+cu118, 11.8


In [16]:
model.to(device)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=3136, out_features=128, bias=True)
  (8): ReLU()
  (9): Linear(in_features=128, out_features=10, bias=True)
)

In [18]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
num_epochs = 6
time = dt.now()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for images, labels in train_loader:
        
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, interval: {dt.now() - time}")

Epoch 1/6, Loss: 0.0159, interval: 0:00:09.712307
Epoch 2/6, Loss: 0.0105, interval: 0:00:19.606749
Epoch 3/6, Loss: 0.0084, interval: 0:00:29.404310
Epoch 4/6, Loss: 0.0084, interval: 0:00:39.145483
Epoch 5/6, Loss: 0.0061, interval: 0:00:48.828008
Epoch 6/6, Loss: 0.0065, interval: 0:00:58.494893


In [20]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 99.16%


In [24]:
# Так, наконец этот танец с бубном завершился
# Видеокарта (моя) слишком древняя, чтобы запустить свежий toolkit, поэтому каскадно перебирал версии и в итоге:
# numpy, torch, torchvision, torchaudio теперь старые
# Исходя из результата, ~200сек (CPU) / ~58сек (GPU) - 1 = ~ 2.45 (245%) прироста скорости при вычислениях на GPU

In [25]:
# Дополнительный Сверточный Слой (Convolutional Layer)

# Сложно:
# Добавление сверточных слоев повышает иерархическую глубину сети,
# обеспечивая экстракцию более семантически богатых и композиционных признаков (compositional features).

# Просто:
# Делает модель "умнее" и позволяет ей распознавать более сложные концепции.
# Например, если первый слой ищет линии, то более глубокий слой, используя эти линии, может найти формы, а еще более глубокий — объекты.
# Увеличивает риск переобучения (overfitting), так как модель становится сложнее.

In [26]:
# Поллинг-слой (Pooling Layer)

# Сложно:
# Поллинг обеспечивает пространственную инвариантность (spatial invariance) и выполняет даунсэмплинг (downsampling) карт признаков,
# тем самым снижая вычислительную нагрузку.

# Просто:
# Делает модель устойчивой к небольшим сдвигам и искажениям во входном изображении.
# Модель фокусируется на наличии признака, а не на его точном местоположении, что улучшает обобщение (generalization) на новых данных.

In [28]:
# Паддинг (Padding)

# Сложно:
# Паддинг контролирует сохранение пространственной размерности (spatial resolution) карт признаков после операции свертки,
# предотвращая потерю информации с краев входного тензора.

# Просто:
# Сохраняет важную информацию по краям изображения.
# Без паддинга края изображения быстро "сжимаются" и их данные игнорируются.
# Паддинг помогает поддерживать стабильную геометрическую структуру сети.

In [29]:
# Дополнительный Полносвязный Слой (FC Layer)

# Сложно:
# Добавление FC-слоев увеличивает сложность пространства решений (decision boundary) на финальном этапе классификации
# и усиливает нелинейность модели.

# Просто:
# Делает окончательное решение о классе более сложным и точным.
# Преобразует извлеченные свертками абстрактные признаки в финальные вероятности классов.
# Существенно увеличивает общее количество параметров модели.